In [6]:
using LaTeXStrings, JuMP, HiGHS, CSV, DataFrames, LinearAlgebra, Printf, Statistics, Symbolics, Clarabel

States $i$: 1 (A), 2 (F), 3 (PO), 4 (MO), 5 (RS)

Minutes $t=0,\ldots, 525600$

Sequences $k$: 1

In [2]:
fname = joinpath(homedir(), "research", "gen_287_2013_state.csv")
data = DataFrame(CSV.File(fname)) #, types=Array{DataType, 1}([Dict{String, Int64, Int64, String}])))
X = data[!, "Minutes"]
y = data[!, "s_t"];

$A = \begin{bmatrix}
A_1\\
A_2\\
A_3\\
A_4\\
A_5
\end{bmatrix}$,
$A_1 = \begin{bmatrix}
a_{1,1} \\
a_{1,2} \\
a_{1,3} \\
a_{1,4} \\
a_{1,5}
\end{bmatrix}$, $\ldots \quad$, $A_5 = \begin{bmatrix}
a_{5,1} \\
a_{5,2} \\
a_{5,3} \\
a_{5,4} \\
a_{5,5}
\end{bmatrix}$

In [3]:
function softplus(model, t, u)
    z = @variable(model, [1:2], lower_bound = 0.0)
    @constraint(model, sum(z) <= 1.0)
    @constraint(model, [u - t, 1, z[1]] in MOI.ExponentialCone())
    @constraint(model, [-t, 1, z[2]] in MOI.ExponentialCone())
end

function build_logit_model(X, y, λ)
    n = size(X)
    n = n[1]
    model = Model()
    @variable(model, θ)
    @variable(model, t[1:n])
    for i in 1:n
        u = -(X[i]' * θ) * y[i]
        softplus(model, t[i], u)
    end
    # Add ℓ2 regularization
    @variable(model, 0.0 <= reg)
    @constraint(model, [reg; θ] in SecondOrderCone())
    # Define objective
    @objective(model, Min, sum(t) + λ * reg)
    return model
end

build_logit_model (generic function with 1 method)

In [4]:
λ = 10.0
model = build_logit_model(X, y, λ)

A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 1576805
├ num_constraints: 2628007
│ ├ AffExpr in MOI.LessThan{Float64}: 525601
│ ├ Vector{VariableRef} in MOI.SecondOrderCone: 1
│ ├ Vector{AffExpr} in MOI.ExponentialCone: 1051202
│ └ VariableRef in MOI.GreaterThan{Float64}: 1051203
└ Names registered in the model
  └ :reg, :t, :θ

In [ ]:
set_optimizer(model, Clarabel.Optimizer)
set_silent(model)
optimize!(model)
assert_is_solved_and_feasible(model)

In [ ]:
θ♯ = value.(model[:θ])